# 업무추진비 보고서

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import re, os
import category_kakao as kakao

In [2]:
file = 'output_data'
files = [i for i in os.listdir(file) if i.endswith('.csv')]
df = pd.read_csv(f'output_data\\{files[-1]}')
df = df.drop(columns=['Unnamed: 0'])

In [3]:
df

,department,name,cost,people,date,category
0,감사위원회,기찬메밀국수,373000,29,2023-04-03,미분류
1,감사위원회,에어산,199500,29,2023-04-03,미분류
2,감사위원회,나루스시,78000,4,2023-04-04,일식
3,감사위원회,석교리,119000,7,2023-04-06,한식
4,감사위원회,순남이네,45000,4,2023-04-11,한식
...,...,...,...,...,...,...
689,환경정책과,신미정춘천닭갈비,86300,6,2023-04-22,한식
690,회계과,오롯이,64000,4,2023-04-04,일식
691,회계과,육산,144000,12,2023-04-12,한식
692,회계과,그릴200도,206000,10,2023-04-20,한식


## 업무 추진비 사용 경향

In [4]:
# 업무 추진비 사용 금액
department_cost = df.groupby('department')['cost'].sum()
print(department_cost)

department
감사위원회     2981400
감염병관리과     476500
건설교통국     1203300
건축과        389000
경제산업국      879000
           ...   
한솔동        305000
해밀동        226900
환경녹지국     1145900
환경정책과      348620
회계과        501000
Name: cost, Length: 100, dtype: int64


In [5]:
# 부서별 인원수 대비 금액
department_people = df.groupby('department')['people'].sum()
department_cost_per_person = department_cost / department_people
print(round(department_cost_per_person), 1)

department
감사위원회     17234.0
감염병관리과    14439.0
건설교통국     12939.0
건축과       13893.0
경제산업국     21439.0
           ...   
한솔동       12200.0
해밀동       14181.0
환경녹지국      6821.0
환경정책과      8300.0
회계과       16700.0
Length: 100, dtype: float64 1


In [6]:
# 가장 많이 방문한 가게
name_count = df['name'].value_counts()
print(name_count)

오롯이        17
궁궐         11
두툼         11
관내업체        8
내토왕코다리      8
           ..
세이로정        1
신미정닭갈비      1
온기          1
다인아구찜       1
우리숯불닭갈비     1
Name: name, Length: 431, dtype: int64


In [7]:
# 가장 많은 금액을 쓴 가게
name_cost = df.groupby('name')['cost'].sum()
name_cost = name_cost.sort_values(ascending=False)
print(name_cost)

name
오롯이          1943000
궁궐           1722400
세종장어대가       1238000
관내업체         1218000
두툼            994500
              ...   
주식회사해궁         12000
텐퍼센트커피어진점      12000
콩카페            11900
커피소녀#(서울)      11000
보비스쿰커피랩        11000
Name: cost, Length: 431, dtype: int64


In [8]:
# 이름별 비용 합계 계산
name_counts = name_count[name_count >= 2]
cost_count = name_cost/name_counts
cost_count = cost_count.dropna()
cost_count = cost_count.sort_values(ascending=False)

In [9]:
# 다양한 부서가 방문한 가게
depart_name = df.groupby('name')['department'].nunique()
depart_name = depart_name.sort_values(ascending=False)

### Markdown 문서 생성

In [10]:
header = f"""
# 세종시 업무추진비 맛집 ({files[-1][:-4]})

업무추진비 URL : https://www.sejong.go.kr/bbs/R0091/list.do
"""

body = f"""
## 업무 추진비 사용 내역

### (1) 많이 방문한 가게

{name_count.head(5).to_markdown()}

### (2) 많은 금액을 지출한 가게

{name_cost.map('{:,.0f}'.format).head(5).to_markdown()}

### (3) 방문 횟수 대비 비용이 큰 가게

{cost_count.map('{:,.0f}'.format).head(5).to_markdown()}

### (4) 다양한 부서가 방문하는 가게

{depart_name.map('{:,.0f}'.format).head(5).to_markdown()}

"""

md_file = header + body

### MD 파일 작성

In [12]:
# 파일 작성
if not os.path.exists('report'):
    os.makedirs('report')

with open(f"report\\{files[-1][:-4]}.md", "w", encoding="utf-8") as f:
  f.write(md_file)